In [3]:
from collections import defaultdict
from math import atan2, degrees

# Day 10: Monitoring Station
Here are some sample maps to test out.

In [4]:
data = [
".#..#",
".....",
"#####",
"....#",
"...##",
]

In [5]:
data = [
"......#.#.",
"#..#.#....",
"..#######.",
".#.#.###..",
".#..#.....",
"..#....#.#",
"#..#....#.",
".##.#..###",
"##...#..#.",
".#....####",
]

In [6]:
with open('day_10.txt') as f:
    data = [row.strip() for row in f.readlines()]

## Part 1: find the roid who can see the most other roids
First, convert the input data into a list of roid coordinate pairs.

In [7]:
roids = []
for j, row in enumerate(data):
    for i, obj in enumerate(row):
        if obj == '#':
            roids.append((i, j))

Next, for each roid, figure out how many other roids it can see. To do this, we normalise each roid's coordinates by dividing by its distance to the roid we're standing on (using Manhattan distance for simplicity). Roids that are hidden behind other roids will map to the same normalized coordinates as the roid they're hiding behind.

Then we just count the number of unique such normalized roids, using a `set` to help.

In [8]:
vis_totals = {}
for roid in roids:
    angles = set()
    for other in roids:
        if roid == other:
            continue
        rise = roid[1] - other[1]
        run = roid[0] - other[0]
        length = abs(rise) + abs(run)
        angles.add((rise/length, run/length)) 
    vis_totals[roid] = len(angles)

The best roid is the one with the highest visibility count.

In [10]:
best_roid, visible_roids = sorted([(k, vis_totals[k]) for k in vis_totals], key=lambda v: v[1])[-1]
print(best_roid, visible_roids)

(22, 25) 286


## Part 2: shoot down the roids
Now we have to figure out how to shoot all the roids in spinning order.

To do this, we'll use a similar technique as before, only instead of mapping each roid to its normalized version, we're going to map it to it's clockwise-angle from 90 degrees (aka 12 o'clock).

Note: to get the angles to work we have to negate the change in `y` from the usual formula. It was also a bit tricky to figure out how to convert the domain of arctan (-pi, pi) to an equivalent domain that is in our desired spinning order.

In [12]:
angles = defaultdict(list)
for other in roids:
    if best_roid == other:
        continue
    y = best_roid[1] - other[1]
    x = other[0] - best_roid[0]
    degs = degrees(atan2(y,x))
    if degs <= 90:
        angle = 90 - degs
    else:
        angle = 90 - degs + 360
    angles[angle].append(other)

Next we'll make a list version of this dict, sorted by angles, and then within each angle, the roids will be sorted by distanece from the `best_roid` (again, we use Manhattan distance for simplicity).

In [18]:
sorted_angles = []
for angle, roids in sorted([(k, angles[k]) for k in angles], key=lambda v: v[0]):
    sorted_angles.append(
        (
            angle, 
            sorted(roids, key=lambda roid: abs(roid[0] - best_roid[0]) + abs(roid[1] - best_roid[1]))
        )
    )

Then we iterate through our `sorted_angles` list, once per 360 degree rotation, popping of one roid at a time until there are none left.

In [19]:
shots = []
while sorted_angles:
    for angle, roids in sorted_angles:
        shots.append(roids.pop(0))
    sorted_angles = [pair for pair in sorted_angles if pair[1]]

The final answer is just the coords of the 200th roid

In [20]:
shots[199]

(5, 4)